In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
import seaborn as sns
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import joblib


In [ ]:
def random_forest(df):
    f_select=[]
   
    X = df.drop('winner',1)
    y = df['winner'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    clf = RandomForestClassifier(n_estimators=100)

    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    proba = clf.predict_proba(X_test)
    proba = np.around(proba, decimals=2)


    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

    # feature importanti per determinarlo
    feature_imp = pd.Series(clf.feature_importances_, index=X.columns).sort_values(ascending=False)
    print('Feature importance:\n', feature_imp)
    f_selection=feature_imp[(feature_imp>0)]
    print('Feature selection:\n',f_selection)
            
        
    # Creating a bar plot
    sns.barplot(x=f_selection, y=f_selection.index)
    # Add labels to your graph
    plt.xlabel('Feature Importance Score')
    plt.ylabel('Features')
    plt.title("Visualizing Important Features")
    plt.legend()
    plt.show()
    #return X_test, y_test, y_pred, proba


In [ ]:
def linear_svm(df):
    X = df.drop('winner',1)
    y = df['winner'].values


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
    svclassifier = SVC(kernel='linear', probability=True)
    svclassifier.fit(X_train, y_train)
    proba = svclassifier.predict_proba(X_test)
    proba = np.around(proba, decimals=2)
    y_pred = svclassifier.predict(X_test)

    print('Confusion matrix:\n', confusion_matrix(y_test, y_pred))
    print('Report:\n', classification_report(y_test, y_pred))
    print('Accuracy:', metrics.accuracy_score(y_test, y_pred))
    print('Proba',proba)

    

In [ ]:
def linear_svm1(df):
    X = df.drop('winner',1)
    y = df['winner'].values


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

    svclassifier =LinearSVC()
    svclassifier.fit(X_train, y_train)
    #proba = svclassifier.predict_proba(X_test)
    #proba = np.around(proba, decimals=2)
    y_pred = svclassifier.predict(X_test)

    print('Confusion matrix:\n', confusion_matrix(y_test, y_pred))
    print('Report:\n', classification_report(y_test, y_pred))
    print('Accuracy:', metrics.accuracy_score(y_test, y_pred))
    #print('Proba',proba)

In [ ]:
def boruta(df):
    X = df.drop('winner',1)
    y = df['winner'].values
    ###initialize Boruta
    forest = RandomForestRegressor(
       n_jobs = -1, 
       max_depth = 5
    )
    boruta = BorutaPy(
       estimator = forest, 
       n_estimators = 'auto',
       max_iter = 100 # number of trials to perform
    )
    ### fit Boruta (it accepts np.array, not pd.DataFrame)
    boruta.fit(np.array(X), np.array(y))
    ### print results
    green_area = X.columns[boruta.support_].to_list()
    blue_area = X.columns[boruta.support_weak_].to_list()
    print('features in the green area:', green_area)
    print('features in the blue area:', blue_area)

In [ ]:
def naive_bayes(df):
    X = df.drop('winner',1)
    y = df['winner'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
    gnb = GaussianNB()
    y_pred = gnb.fit(X_train, y_train).predict(X_test)
    print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (y_test != y_pred).sum()))
    print('Confusion matrix:\n', confusion_matrix(y_test, y_pred))
    print('Report:\n', classification_report(y_test, y_pred))
    print('Accuracy:', metrics.accuracy_score(y_test, y_pred))

In [3]:
df = pd.read_pickle('../../data.scenarioTest2v2.pkl.gz')
#df=pd.read_csv('../docs/csv/csvscenarioJunction.csv')

In [ ]:
c=df.select_dtypes(include=['object']).drop(['winner'], axis=1).columns


# Label Encoder


In [ ]:
le_df=df
le = preprocessing.LabelEncoder()
le_df[c] = df[c].apply(lambda col: le.fit_transform(col))

# One Hot Encoder

In [ ]:
ohe_df=df
enc = OneHotEncoder(handle_unknown='ignore')
# passing bridge-types-cat column (label encoded values of bridge_types)
enc_df = pd.DataFrame(enc.fit_transform(ohe_df[c]).toarray())
# merge with main df bridge_df on key values
ohe_df = ohe_df.join(enc_df)
ohe_df

In [ ]:
lb = preprocessing.LabelBinarizer()
print ("Label Binarizer:", lb.fit_transform(df['team_red_2_0']))

In [ ]:
lb_df=df
lb = preprocessing.LabelBinarizer()
lb_df[c] = df[c].apply(lambda col: lb.fit_transform(col))


# Dummies

In [ ]:
bridge_df=df
X = pd.get_dummies(bridge_df, prefix_sep='_', drop_first=True)
bridge_df = bridge_df.merge(X)
bridge_df

In [ ]:
X

In [ ]:
plt.scatter(df.index,df['kind_red_Tank1'])

In [ ]:
random_forest(le_df)

In [ ]:
boruta(df)

linear_svm(df)

In [ ]:
linear_svm1(le_df)

In [ ]:
naive_bayes(le_df)

In [ ]:
linear_svm(df)